In [2]:
import requests
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

In [5]:
test = {"City": ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"], "State": ["NY", "CA", "IL", "TX", "AZ", "PA", "TX", "CA", "TX", "CA"]}
test_df = pd.DataFrame(data=test)
test_df 

,City,State
0,New York,NY
1,Los Angeles,CA
2,Chicago,IL
3,Houston,TX
4,Phoenix,AZ
5,Philadelphia,PA
6,San Antonio,TX
7,San Diego,CA
8,Dallas,TX
9,San Jose,CA


In [6]:
test_df .replace(' ', '+',regex=True,inplace=True)
test_df

,City,State
0,New+York,NY
1,Los+Angeles,CA
2,Chicago,IL
3,Houston,TX
4,Phoenix,AZ
5,Philadelphia,PA
6,San+Antonio,TX
7,San+Diego,CA
8,Dallas,TX
9,San+Jose,CA


In [8]:
test_df['var'] = test_df.agg('{0[City]}%2C+{0[State]}'.format, axis=1)
test_df

,City,State,var
0,New+York,NY,New+York%2C+NY
1,Los+Angeles,CA,Los+Angeles%2C+CA
2,Chicago,IL,Chicago%2C+IL
3,Houston,TX,Houston%2C+TX
4,Phoenix,AZ,Phoenix%2C+AZ
5,Philadelphia,PA,Philadelphia%2C+PA
6,San+Antonio,TX,San+Antonio%2C+TX
7,San+Diego,CA,San+Diego%2C+CA
8,Dallas,TX,Dallas%2C+TX
9,San+Jose,CA,San+Jose%2C+CA


In [14]:
var = pd.Series(test_df['var'])

In [3]:
csv_path = "covid_data/DS-C-19-COUNTIES-1.csv"
cov_counties_1_df  = pd.read_csv(csv_path, encoding="utf-8")

In [5]:
cov_counties_1_df.head()

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018,N_POP_CHG_2018,Births_2018,Deaths_2018,...,Total number of UCR (Uniform Crime Report) Index crimes excluding arson.,Total number of UCR (Uniform Crime Report) index crimes reported including arson,MURDER,RAPE,ROBBERY,Number of AGGRAVATED ASSAULTS,BURGLRY,LARCENY,MOTOR VEHICLE THEFTS,ARSON
0,0,US,United States,NaN,NaN,NaN,327167434,2020313.0,3855500.0,2814013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,4887871,12751.0,57216.0,53425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,AL,Autauga County,2.0,2.0,0.0,55601,158.0,655.0,532.0,...,139.0,1765.0,3.0,15.0,34.0,87.0,447.0,1233.0,85.0,108.0
3,1003,AL,Baldwin County,3.0,2.0,5.0,218022,5403.0,2254.0,2228.0,...,446.0,4988.0,8.0,30.0,76.0,332.0,967.0,3829.0,192.0,31.0
4,1005,AL,Barbour County,6.0,6.0,3.0,24881,-277.0,261.0,324.0,...,48.0,474.0,0.0,4.0,8.0,36.0,90.0,362.0,21.0,0.0


In [7]:
clean_county1_df = cov_counties_1_df[['Area_Name', 'State', 'POP_ESTIMATE_2018', 'Density per square mile of land area - Population', 'Unemployment_rate_2018', 'Median_Household_Income_2018', 'Total_Male', 'Total_Female', 'Total_age0to17', 'Male_age0to17', 'Female_age0to17', 'Total_age18to64', 'Male_age18to64', 'Female_age18to64', 'Total_age65plus', 'Male_age65plus', 'Female_age65plus', 'Total_age85plusr', 'Male_age85plusr', 'Female_age85plusr', 'ICU Beds', 'transit_scores - population weighted averages aggregated from town/city level to county']]
clean_county1_df

,Area_Name,State,POP_ESTIMATE_2018,Density per square mile of land area - Population,Unemployment_rate_2018,Median_Household_Income_2018,Total_Male,Total_Female,Total_age0to17,Male_age0to17,...,Male_age18to64,Female_age18to64,Total_age65plus,Male_age65plus,Female_age65plus,Total_age85plusr,Male_age85plusr,Female_age85plusr,ICU Beds,transit_scores - population weighted averages aggregated from town/city level to county
0,United States,US,327167434,87.4,3.9,61937.0,161128679.0,166038755.0,73399342.0,37490849.0,...,100331012.0,101005887.0,52431193.0,23306818.0,29124375.0,6544503.0,2325693.0,4218810.0,NaN,NaN
1,Alabama,AL,4887871,94.4,3.9,49881.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga County,AL,55601,91.8,3.6,59338.0,26995.0,28606.0,13163.0,6726.0,...,16482.0,17303.0,8653.0,3787.0,4866.0,871.0,329.0,542.0,6.0,0.0
3,Baldwin County,AL,218022,114.6,3.6,57588.0,105657.0,112365.0,47110.0,23810.0,...,61166.0,65175.0,44571.0,20681.0,23890.0,4304.0,1755.0,2549.0,51.0,368329051.0
4,Barbour County,AL,24881,31.0,5.2,34382.0,13133.0,11748.0,5192.0,2630.0,...,8414.0,6443.0,4832.0,2089.0,2743.0,472.0,148.0,324.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3268,Vega Baja Municipio Puerto Rico,PR,50185,1301.1,10.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3269,Vieques Municipio Puerto Rico,PR,8364,183.2,16.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,Villalba Municipio Puerto Rico,PR,21476,731.6,15.9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3271,Yabucoa Municipio Puerto Rico,PR,32158,687.2,14.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clean_county1_df.rename(columns={
                                }, inplace=True)